In [59]:
import os
import pickle
import sys

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, roc_auc_score, average_precision_score, f1_score, accuracy_score, roc_curve
from sklearn.metrics import precision_recall_curve
from scipy import stats

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model

from hnet import AppendNet

def load_pretrained_model(pre_trained_loc="./PCLR.h5") :
    pre_trained_model = load_model(pre_trained_loc)
    
    return pre_trained_model

def do_bootstrap_regression(preds, trues, n=1000):
    rmse_list = []
    r_list = []
    pval_list = []

    rng = np.random.RandomState(seed=1)
    for _ in range(n):
        idxs = rng.choice(len(trues), size=len(trues), replace=True)
        pred_arr = preds[idxs]
        true_arr = trues[idxs]

        rmse = rmse_loss(pred_arr, true_arr)
        r, pval = stats.pearsonr(true_arr, pred_arr)

        rmse_list.append(rmse)
        r_list.append(r)
        pval_list.append(pval)

    return np.array(rmse_list), np.array(r_list), np.array(pval_list)

def confidence_interval(values, alpha=0.95):
    lower = np.percentile(values, (1-alpha)/2 * 100)
    upper = np.percentile(values, (alpha + (1-alpha)/2) * 100)
    return lower, upper

In [60]:
def get_ecg(df):
    ecgs = []
    for idx in df.index:
        row = df.loc[idx]
        qid = row['QuantaID']
        doc = row['Date_of_Cath']
        fname = f'/storage/shared/apollo/same-day/{qid}_{doc}.csv'
        x = pd.read_csv(fname).values[...,1:].astype(np.float32)
        x /= 1000
        x = x[:4096, :].T
        ecgs.append(x)
        
    ecgs = np.array(ecgs)
    return np.transpose(ecgs, (0,2,1))

def get_data(batch_size=64):
    df_tab = pd.read_csv(os.path.join('/storage/shared/apollo/same-day/tabular_data.csv'))
    train_ids = np.load("./stores/train_ids.npy")
    val_ids = np.load("./stores/val_ids.npy")
    test_ids = np.load("./stores/test_ids.npy")

    train_ids = train_ids[len(train_ids) // 2 :]
    val_ids = val_ids[len(val_ids) // 2 :]
    test_ids = test_ids[len(test_ids) // 2 :]

    train_df = df_tab[df_tab["QuantaID"].isin(train_ids)]
    val_df = df_tab[df_tab["QuantaID"].isin(val_ids)]
    test_df = df_tab[df_tab["QuantaID"].isin(test_ids)]
    print(len(train_df), len(val_df), len(test_df))

    X_train = get_ecg(train_df)
    X_val = get_ecg(val_df)
    X_test = get_ecg(test_df)

    y_train = train_df["PCWP_mean"].values
    y_val = val_df["PCWP_mean"].values
    y_test = test_df["PCWP_mean"].values

    return X_train, y_train, X_val, y_val, X_test, y_test

def rmse_loss(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

In [61]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

pre_trained_model = load_pretrained_model(pre_trained_loc='./PCLR.h5')
latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)
full_model = AppendNet(latent, new_layers = [128, 1], classification=False) # same hidden dimension as dml (128)

optimizer = tf.keras.optimizers.Adam() # can modify LR, of course
# loss_fn = tf.keras.losses.BinaryCrossentropy()
loss_fn = rmse_loss

X_train, y_train, X_val, y_val, X_test, y_test = get_data()

2442 893 923


In [25]:
epochs = 50
full_model.compile(optimizer, loss_fn)
full_model.fit(X_train, y_train, epochs=epochs)

Epoch 1/50
77/77 [==============================] - 12s 99ms/step - loss: 15.0063
Epoch 2/50
77/77 [==============================] - 8s 103ms/step - loss: 13.9235
Epoch 3/50
77/77 [==============================] - 11s 147ms/step - loss: 13.7081
Epoch 4/50
77/77 [==============================] - 8s 103ms/step - loss: 13.2645
Epoch 5/50
77/77 [==============================] - 8s 102ms/step - loss: 13.6062
Epoch 6/50
77/77 [==============================] - 8s 103ms/step - loss: 13.4615
Epoch 7/50
77/77 [==============================] - 8s 104ms/step - loss: 13.6717
Epoch 8/50
77/77 [==============================] - 8s 103ms/step - loss: 13.6344
Epoch 9/50
77/77 [==============================] - 8s 99ms/step - loss: 13.0034
Epoch 10/50
77/77 [==============================] - 8s 99ms/step - loss: 13.3835
Epoch 11/50
77/77 [==============================] - 8s 100ms/step - loss: 13.4399
Epoch 12/50
77/77 [==============================] - 8s 100ms/step - loss: 13.2835
Epoch 13/50
77

In [46]:
epochs = 100
pre_trained_model = load_pretrained_model(pre_trained_loc='./PCLR.h5')
latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)
full_model = AppendNet(latent, new_layers = [128, 1], classification=False)

optimizer = tf.keras.optimizers.Adam() # can modify LR, of course
# loss_fn = tf.keras.losses.BinaryCrossentropy()
loss_fn = rmse_loss
full_model.compile(optimizer, loss_fn)
full_model.fit(X_train, y_train, epochs=epochs)

Epoch 1/100
77/77 [==============================] - 12s 98ms/step - loss: 15.5466
Epoch 2/100
77/77 [==============================] - 8s 102ms/step - loss: 14.0876
Epoch 3/100
77/77 [==============================] - 8s 101ms/step - loss: 13.7544
Epoch 4/100
77/77 [==============================] - 8s 101ms/step - loss: 13.3075
Epoch 5/100
77/77 [==============================] - 8s 104ms/step - loss: 13.6482
Epoch 6/100
77/77 [==============================] - 8s 103ms/step - loss: 13.4675
Epoch 7/100
77/77 [==============================] - 8s 107ms/step - loss: 13.6814
Epoch 8/100
77/77 [==============================] - 8s 102ms/step - loss: 13.6239
Epoch 9/100
77/77 [==============================] - 8s 103ms/step - loss: 13.0276
Epoch 10/100
77/77 [==============================] - 12s 153ms/step - loss: 13.3434
Epoch 11/100
77/77 [==============================] - 8s 101ms/step - loss: 13.4667
Epoch 12/100
77/77 [==============================] - 8s 102ms/step - loss: 13.3233


In [51]:
epochs = 150
pre_trained_model = load_pretrained_model(pre_trained_loc='./PCLR.h5')
latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)
full_model = AppendNet(latent, new_layers = [128, 1], classification=False)

optimizer = tf.keras.optimizers.Adam() # can modify LR, of course
# loss_fn = tf.keras.losses.BinaryCrossentropy()
loss_fn = rmse_loss
full_model.compile(optimizer, loss_fn)
full_model.fit(X_train, y_train, epochs=epochs)

Epoch 1/150
77/77 [==============================] - 12s 104ms/step - loss: 14.9223
Epoch 2/150
77/77 [==============================] - 8s 100ms/step - loss: 13.9433
Epoch 3/150
77/77 [==============================] - 8s 103ms/step - loss: 13.6449
Epoch 4/150
77/77 [==============================] - 8s 105ms/step - loss: 13.2819
Epoch 5/150
77/77 [==============================] - 8s 104ms/step - loss: 13.5724
Epoch 6/150
77/77 [==============================] - 8s 108ms/step - loss: 13.4860
Epoch 7/150
77/77 [==============================] - 8s 107ms/step - loss: 13.6422
Epoch 8/150
77/77 [==============================] - 8s 106ms/step - loss: 13.6212
Epoch 9/150
77/77 [==============================] - 8s 107ms/step - loss: 13.0164
Epoch 10/150
77/77 [==============================] - 12s 154ms/step - loss: 13.3576
Epoch 11/150
77/77 [==============================] - 8s 105ms/step - loss: 13.4080
Epoch 12/150
77/77 [==============================] - 8s 106ms/step - loss: 13.3292

In [62]:
epochs = 200
pre_trained_model = load_pretrained_model(pre_trained_loc='./PCLR.h5')
latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)
full_model = AppendNet(latent, new_layers = [128, 1], classification=False)

optimizer = tf.keras.optimizers.Adam() # can modify LR, of course
# loss_fn = tf.keras.losses.BinaryCrossentropy()
loss_fn = rmse_loss
full_model.compile(optimizer, loss_fn)
full_model.fit(X_train, y_train, epochs=epochs)

Epoch 1/200
77/77 [==============================] - 12s 96ms/step - loss: 15.4870
Epoch 2/200
77/77 [==============================] - 8s 98ms/step - loss: 14.0569
Epoch 3/200
77/77 [==============================] - 8s 98ms/step - loss: 13.7952
Epoch 4/200
77/77 [==============================] - 7s 97ms/step - loss: 13.4087
Epoch 5/200
77/77 [==============================] - 7s 95ms/step - loss: 13.6161
Epoch 6/200
77/77 [==============================] - 7s 95ms/step - loss: 13.4954
Epoch 7/200
77/77 [==============================] - 7s 93ms/step - loss: 13.7052
Epoch 8/200
77/77 [==============================] - 7s 95ms/step - loss: 13.6811
Epoch 9/200
77/77 [==============================] - 10s 126ms/step - loss: 13.0129
Epoch 10/200
77/77 [==============================] - 8s 98ms/step - loss: 13.3911
Epoch 11/200
77/77 [==============================] - 7s 97ms/step - loss: 13.3794
Epoch 12/200
77/77 [==============================] - 7s 96ms/step - loss: 13.3289
Epoch 13/2

77/77 [==============================] - 7s 92ms/step - loss: 12.9758
Epoch 100/200
77/77 [==============================] - 7s 93ms/step - loss: 12.9770
Epoch 101/200
77/77 [==============================] - 7s 94ms/step - loss: 12.8791
Epoch 102/200
77/77 [==============================] - 7s 91ms/step - loss: 12.6094
Epoch 103/200
77/77 [==============================] - 7s 91ms/step - loss: 12.9395
Epoch 104/200
77/77 [==============================] - 7s 92ms/step - loss: 12.8842
Epoch 105/200
77/77 [==============================] - 7s 92ms/step - loss: 13.0374
Epoch 106/200
77/77 [==============================] - 7s 91ms/step - loss: 12.4947
Epoch 107/200
77/77 [==============================] - 7s 91ms/step - loss: 12.7363
Epoch 108/200
77/77 [==============================] - 7s 94ms/step - loss: 12.6492
Epoch 109/200
77/77 [==============================] - 7s 92ms/step - loss: 12.8198
Epoch 110/200
77/77 [==============================] - 7s 93ms/step - loss: 12.7683
Epoch 

77/77 [==============================] - 7s 92ms/step - loss: 12.5300
Epoch 197/200
77/77 [==============================] - 7s 93ms/step - loss: 12.2884
Epoch 198/200
77/77 [==============================] - 7s 91ms/step - loss: 12.6437
Epoch 199/200
77/77 [==============================] - 7s 91ms/step - loss: 12.5554
Epoch 200/200
77/77 [==============================] - 7s 92ms/step - loss: 12.6519


In [63]:
'''
Getting prediction result from test set
'''
y_pred = full_model.predict(X_test)

29/29 [==============================] - 2s 35ms/step


In [50]:
full_model.save('./PCLR_finetuned_100epc.pb', save_format='tf')

INFO:tensorflow:Assets written to: ./PCLR_finetuned_100epc.pb/assets


INFO:tensorflow:Assets written to: ./PCLR_finetuned_100epc.pb/assets


In [ ]:
# Load saved tf model
loaded_model = tf.keras.models.load_model('./PCLR_finetuned.pb')

# Calculate Classification Performance

In [38]:
from scipy import stats
r, pval = stats.pearsonr(y_test, np.concatenate(y_pred))
print(rmse_loss(y_test, y_pred))
print(r, pval)

tf.Tensor(11.718298284532933, shape=(), dtype=float64)
0.5179734023757416 1.7000212654359237e-64


In [45]:
'''
50 epoch model
'''
rmse, r, pval = do_bootstrap_regression(np.concatenate(y_pred), y_test)
print(rmse.mean(), r.mean(), pval.mean())
print(confidence_interval(rmse), confidence_interval(r), confidence_interval(pval))

10.662241673856977 0.5168803322877793 3.885793744767385e-48
(10.168321327256356, 11.186809058909072) (0.46935757473184414, 0.5653645253170076) (4.403350490234938e-79, 9.551261205355765e-52)


In [49]:
'''
100 epoch model
'''
rmse, r, pval = do_bootstrap_regression(np.concatenate(y_pred), y_test)
print(rmse.mean(), r.mean(), pval.mean())
print(confidence_interval(rmse), confidence_interval(r), confidence_interval(pval))

10.08667670973824 0.5084383288648698 5.124600599466284e-44
(9.593113688193553, 10.608999002632787) (0.4534978599475333, 0.5592618550246696) (4.559193565980286e-77, 5.1785051956098e-48)


In [55]:
'''
150 epoch model
'''
rmse, r, pval = do_bootstrap_regression(np.concatenate(y_pred), y_test)
print(rmse.mean(), r.mean(), pval.mean())
print(confidence_interval(rmse), confidence_interval(r), confidence_interval(pval))

9.848489722993618 0.46944465590783035 3.0449579264607663e-35
(9.338509122190928, 10.366695124278358) (0.4179658931128352, 0.5250832141152932) (1.53915223843851e-66, 2.4962666858646983e-40)


In [64]:
'''
200 epoch model
'''
rmse, r, pval = do_bootstrap_regression(np.concatenate(y_pred), y_test)
print(rmse.mean(), r.mean(), pval.mean())
print(confidence_interval(rmse), confidence_interval(r), confidence_interval(pval))

10.292663228474936 0.47730382296197216 2.970788508126865e-37
(9.783613390282376, 10.830710205153133) (0.42562339353459316, 0.5319061016216475) (1.5150117569838069e-68, 6.574814400434942e-42)
